In [1]:
import ipynb.fs.full.helper as h

from imp import reload
reload(h)

<module 'ipynb.fs.full.helper' (/tf/src/scripts/helper.ipynb)>

In [2]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [3]:
spec = model_spec.get('efficientdet_lite0') # maybe different lite1-4
#spec.model_dir="/logs/fit/test/"

In [15]:
def read_train_val_test(dataset_dir, label_map):
    return tuple(map(lambda set_: object_detector.DataLoader.from_pascal_voc(images_dir=os.path.join(dataset_dir, set_), annotations_dir=os.path.join(dataset_dir, set_), label_map=label_map), ("train", "val", "test")))

In [16]:
train_data, validation_data, test_data = read_train_val_test(h.datasetsTf, h.query_array)

In [10]:
model = object_detector.create(train_data, model_spec=spec, batch_size=16, train_whole_model=True, validation_data=validation_data, epochs=5)

Epoch 1/5
1/1 [==============================] - 19s 19s/step - det_loss: 2.6198 - cls_loss: 1.8633 - box_loss: 0.0151 - reg_l2_loss: 0.0757 - loss: 2.6955 - learning_rate: 0.0080 - gradient_norm: 1.5450
Epoch 2/5
1/1 [==============================] - 1s 974ms/step - det_loss: 2.5765 - cls_loss: 1.8437 - box_loss: 0.0147 - reg_l2_loss: 0.0757 - loss: 2.6522 - learning_rate: 0.0171 - gradient_norm: 1.3236
Epoch 3/5
1/1 [==============================] - 1s 980ms/step - det_loss: 2.4815 - cls_loss: 1.7989 - box_loss: 0.0137 - reg_l2_loss: 0.0757 - loss: 2.5571 - learning_rate: 0.0100 - gradient_norm: 1.2681
Epoch 4/5
1/1 [==============================] - 1s 978ms/step - det_loss: 2.4235 - cls_loss: 1.6821 - box_loss: 0.0148 - reg_l2_loss: 0.0757 - loss: 2.4992 - learning_rate: 0.0029 - gradient_norm: 1.0581
Epoch 5/5
1/1 [==============================] - 1s 975ms/step - det_loss: 2.3950 - cls_loss: 1.6809 - box_loss: 0.0143 - reg_l2_loss: 0.0757 - loss: 2.4707 - learning_rate: 0.0000e

In [11]:
model.evaluate(test_data)

1/1 [==============================] - 3s 3s/step



{'AP': 0.0,
 'AP50': 0.0,
 'AP75': 0.0,
 'APs': -1.0,
 'APm': -1.0,
 'APl': 0.0,
 'ARmax1': 0.0,
 'ARmax10': 0.0,
 'ARmax100': 0.0,
 'ARs': -1.0,
 'ARm': -1.0,
 'ARl': 0.0,
 'AP_/LewisHamilton': 0.0,
 'AP_/SebastianVettel': -1.0,
 'AP_/MaxVerstappen': -1.0,
 'AP_/KarlLauterbach': -1.0,
 'AP_/CristianoRonaldo': -1.0,
 'AP_/LionelMessi': -1.0,
 'AP_/EdSheeran': -1.0,
 'AP_/TomCruise': -1.0,
 'AP_/HeleneFischer': -1.0,
 'AP_/ArnoldSchwarzenegger': -1.0,
 'AP_/GüntherJauch': -1.0,
 'AP_/StefanRaab': -1.0,
 'AP_/HeidiKlum': -1.0,
 'AP_/ManuelNeuer': -1.0,
 "AP_/ElyasM'barek": -1.0,
 'AP_/JudithRakers': -1.0,
 'AP_/AngelaMerkel': -1.0,
 'AP_/WladimirPutin': -1.0,
 'AP_/OlafScholz': -1.0,
 'AP_/JoeBiden': -1.0,
 'AP_/DonaldTrump': -1.0,
 'AP_/ChristianLindner': -1.0,
 'AP_/AnnalenaBaerbock': -1.0,
 'AP_/ElonMusk': -1.0,
 'AP_/JeffBezos': -1.0,
 'AP_/BarackObama': -1.0,
 'AP_/ChrisPratt': -1.0,
 'AP_/XiJinping': -1.0,
 'AP_/DwayneJohnson': -1.0,
 'AP_/MacKenzieScott': -1.0,
 'AP_/AnthonyFauci'

In [12]:
model.export(export_dir=h.modelsVTf)

In [13]:
model.evaluate_tflite(h.modelsVTf + '/model.tflite', validation_data)

4/4 [==============================] - 6s 1s/step



{'AP': 0.0,
 'AP50': 0.0,
 'AP75': 0.0,
 'APs': -1.0,
 'APm': -1.0,
 'APl': 0.0,
 'ARmax1': 0.0,
 'ARmax10': 0.0,
 'ARmax100': 0.0,
 'ARs': -1.0,
 'ARm': -1.0,
 'ARl': 0.0,
 'AP_/LewisHamilton': 0.0,
 'AP_/SebastianVettel': 0.0,
 'AP_/MaxVerstappen': 0.0,
 'AP_/KarlLauterbach': 0.0,
 'AP_/CristianoRonaldo': -1.0,
 'AP_/LionelMessi': -1.0,
 'AP_/EdSheeran': -1.0,
 'AP_/TomCruise': -1.0,
 'AP_/HeleneFischer': -1.0,
 'AP_/ArnoldSchwarzenegger': -1.0,
 'AP_/GüntherJauch': -1.0,
 'AP_/StefanRaab': -1.0,
 'AP_/HeidiKlum': -1.0,
 'AP_/ManuelNeuer': -1.0,
 "AP_/ElyasM'barek": -1.0,
 'AP_/JudithRakers': -1.0,
 'AP_/AngelaMerkel': -1.0,
 'AP_/WladimirPutin': -1.0,
 'AP_/OlafScholz': -1.0,
 'AP_/JoeBiden': -1.0,
 'AP_/DonaldTrump': -1.0,
 'AP_/ChristianLindner': -1.0,
 'AP_/AnnalenaBaerbock': -1.0,
 'AP_/ElonMusk': -1.0,
 'AP_/JeffBezos': -1.0,
 'AP_/BarackObama': -1.0,
 'AP_/ChrisPratt': -1.0,
 'AP_/XiJinping': -1.0,
 'AP_/DwayneJohnson': -1.0,
 'AP_/MacKenzieScott': -1.0,
 'AP_/AnthonyFauci': -